### import the libraries we need

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Get the website

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
source.status_code

200

### Scraping the table from the website and put it into a dataframe

In [4]:
soup = BeautifulSoup(source.content,'lxml')
type(soup)

bs4.BeautifulSoup

In [5]:
table = soup.find_all('table')[0]

In [6]:
df = pd.read_html(str(table))

In [7]:
dataframe = df[0]

In [8]:
dataframe.columns.values 

array([0, 1, 2])

In [9]:
dataframe.columns = dataframe.iloc[0]
dataframe.head()

0  Postcode       Borough     Neighbourhood
0  Postcode       Borough     Neighbourhood
1       M1A  Not assigned      Not assigned
2       M2A  Not assigned      Not assigned
3       M3A    North York         Parkwoods
4       M4A    North York  Victoria Village

In [10]:
dataframe = dataframe.reindex(dataframe.index.drop(0))


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [11]:
dataframe = dataframe[dataframe.Borough != 'Not assigned']

In [12]:
print(dataframe.shape)
dataframe.head()

(212, 3)


0 Postcode           Borough     Neighbourhood
3      M3A        North York         Parkwoods
4      M4A        North York  Victoria Village
5      M5A  Downtown Toronto      Harbourfront
6      M5A  Downtown Toronto       Regent Park
7      M6A        North York  Lawrence Heights

In [13]:
dataframe = dataframe.reset_index(drop=True)
print(dataframe.shape)
dataframe.head()

(212, 3)


0 Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M5A  Downtown Toronto       Regent Park
4      M6A        North York  Lawrence Heights

### Combine the rows that has the same postal code, and put the neighbours in the same cell. 

In [14]:
#a = dataframe['Neighbourhood'].groupby(dataframe['Postcode']).apply(lambda x: "{%s}" %', '.join(x))
a = dataframe['Neighbourhood'].groupby([dataframe['Postcode'],dataframe['Borough']]).apply(lambda x: ', '.join(x))
a.head()


Postcode  Borough    
M1B       Scarborough                            Rouge, Malvern
M1C       Scarborough    Highland Creek, Rouge Hill, Port Union
M1E       Scarborough         Guildwood, Morningside, West Hill
M1G       Scarborough                                    Woburn
M1H       Scarborough                                 Cedarbrae
Name: Neighbourhood, dtype: object

In [15]:
df = pd.DataFrame(a).reset_index()
df.head()

Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae

### If a cell has borough but a Not Assigned neighborhood, then the neighborhood will be the same as the borough

In [16]:
a = df[df.Neighbourhood == 'Not assigned'].index
a

Int64Index([85], dtype='int64')

In [17]:
df.loc[a,'Neighbourhood'] = df.loc[a,'Borough']

### print the number of rows of your dataframe.

In [18]:
df.shape

(103, 3)

# Part 2 Add Latitude and longitude to the dataframe

In [19]:
df_ = pd.read_csv('/Users/bowenliu/Documents/ibm_data_scientist/capstone_project/Geospatial_Coordinates.csv')

In [20]:
df_.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [21]:
df_.head()

Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711
3      M1G  43.770992 -79.216917
4      M1H  43.773136 -79.239476

In [22]:
df = pd.merge(df,df_)

In [23]:
df.head()

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

## Explore and cluster the neighborhoods in Toronto. 

In [24]:
import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### work with only boroughs that contain the word Toronto

In [25]:
df = df[df['Borough'].str.contains('Toronto')]
df.reset_index()

index Postcode           Borough  \
0      37      M4E      East Toronto   
1      41      M4K      East Toronto   
2      42      M4L      East Toronto   
3      43      M4M      East Toronto   
4      44      M4N   Central Toronto   
5      45      M4P   Central Toronto   
6      46      M4R   Central Toronto   
7      47      M4S   Central Toronto   
8      48      M4T   Central Toronto   
9      49      M4V   Central Toronto   
10     50      M4W  Downtown Toronto   
11     51      M4X  Downtown Toronto   
12     52      M4Y  Downtown Toronto   
13     53      M5A  Downtown Toronto   
14     54      M5B  Downtown Toronto   
15     55      M5C  Downtown Toronto   
16     56      M5E  Downtown Toronto   
17     57      M5G  Downtown Toronto   
18     58      M5H  Downtown Toronto   
19     59      M5J  Downtown Toronto   
20     60      M5K  Downtown Toronto   
21     61      M5L  Downtown Toronto   
22     63      M5N   Central Toronto   
23     64      M5P   Central Toronto   
24     65      M5R   Central Toronto   
25     66      M5S  Downtown Toronto   
26     67      M5T  Downtown Toronto   
27     68      M5V  Downtown Toronto   
28     69      M5W  Downtown Toronto   
29     70      M5X  Downtown Toronto   
30     75      M6G  Downtown Toronto   
31     76      M6H      West Toronto   
32     77      M6J      West Toronto   
33     78      M6K      West Toronto   
34     82      M6P      West Toronto   
35     83      M6R      West Toronto   
36     84      M6S      West Toronto   
37     87      M7Y      East Toronto   

                                        Neighbourhood   Latitude  Longitude  
0                                         The Beaches  43.676357 -79.293031  
1                        The Danforth West, Riverdale  43.679557 -79.352188  
2                      The Beaches West, India Bazaar  43.668999 -79.315572  
3                                     Studio District  43.659526 -79.340923  
4                                       Lawrence Park  43.728020 -79.388790  
5                                    Davisville North  43.712751 -79.390197  
6                                  North Toronto West  43.715383 -79.405678  
7                                          Davisville  43.704324 -79.388790  
8                         Moore Park, Summerhill East  43.689574 -79.383160  
9   Deer Park, Forest Hill SE, Rathnelly, South Hi...  43.686412 -79.400049  
10                                           Rosedale  43.679563 -79.377529  
11                        Cabbagetown, St. James Town  43.667967 -79.367675  
12                               Church and Wellesley  43.665860 -79.383160  
13                          Harbourfront, Regent Park  43.654260 -79.360636  
14                           Ryerson, Garden District  43.657162 -79.378937  
15                                     St. James Town  43.651494 -79.375418  
16                                        Berczy Park  43.644771 -79.373306  
17                                 Central Bay Street  43.657952 -79.387383  
18                           Adelaide, King, Richmond  43.650571 -79.384568  
19  Harbourfront East, Toronto Islands, Union Station  43.640816 -79.381752  
20           Design Exchange, Toronto Dominion Centre  43.647177 -79.381576  
21                     Commerce Court, Victoria Hotel  43.648198 -79.379817  
22                                           Roselawn  43.711695 -79.416936  
23                Forest Hill North, Forest Hill West  43.696948 -79.411307  
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678  
25                     Harbord, University of Toronto  43.662696 -79.400049  
26          Chinatown, Grange Park, Kensington Market  43.653206 -79.400049  
27  CN Tower, Bathurst Quay, Island airport, Harbo...  43.628947 -79.394420  
28                    Stn A PO Boxes 25 The Esplanade  43.646435 -79.374846  
29             First Canadian Place, Underground city  43.648429 -79.382280  
30                       

## Create a map of Toronto with neighborhoods superimposed on top

In [26]:
latitude = 43.657952
longitude = -79.387383

In [27]:
# create map of toronto using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare Credentials and Version

In [28]:
CLIENT_ID = 'P1E2EGPIORWJ2JCIUBTJ42T41UGN1YSGGJGQONGBJMPZREBX' # your Foursquare ID
CLIENT_SECRET = 'D1PT5043NRXZFD2MYHDWLAAQOBF2IWCL5UQ0AKWEVFNTAFI4' # your Foursquare Secret
VERSION = '20181124' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P1E2EGPIORWJ2JCIUBTJ42T41UGN1YSGGJGQONGBJMPZREBX
CLIENT_SECRET:D1PT5043NRXZFD2MYHDWLAAQOBF2IWCL5UQ0AKWEVFNTAFI4


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [29]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe called toronto_venues that includes Venues information

In [30]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(1713, 7)


Neighborhood  Neighborhood Latitude  \
0                   The Beaches              43.676357   
1                   The Beaches              43.676357   
2                   The Beaches              43.676357   
3                   The Beaches              43.676357   
4  The Danforth West, Riverdale              43.679557   

   Neighborhood Longitude                Venue  Venue Latitude  \
0              -79.293031  Grover Pub and Grub       43.679181   
1              -79.293031            Starbucks       43.678798   
2              -79.293031               My Bbq       43.676881   
3              -79.293031        Upper Beaches       43.680563   
4              -79.352188             Pantheon       43.677621   

   Venue Longitude    Venue Category  
0       -79.297215               Pub  
1       -79.298045       Coffee Shop  
2       -79.289286         BBQ Joint  
3       -79.292869      Neighborhood  
4       -79.351434  Greek Restaurant

#### Let's find out how many unique categories can be curated from all the returned venues

In [59]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} unique Neighbourhood'.format(len(toronto_venues['Neighborhood'].unique())))

There are 241 unique categories.
There are 38 unique Neighbourhood


## Analyze Each Neighbourhood

In [53]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'],prefix="",prefix_sep="")

#add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood']
print(toronto_onehot.shape)
#move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

(1713, 242)


Neighbourhood  Accessories Store  Adult Boutique  \
0                   The Beaches                  0               0   
1                   The Beaches                  0               0   
2                   The Beaches                  0               0   
3                   The Beaches                  0               0   
4  The Danforth West, Riverdale                  0               0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                  0        0                   0             0   
1                  0        0                   0             0   
2                  0        0                   0             0   
3                  0        0                   0             0   
4                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Arepa Restaurant  Argentinian Restaurant  \
0             0         0                 0                       0   
1             0         0                 0                       0   
2             0         0                 0                       0   
3             0         0                 0                       0   
4             0         0                 0                       0   

   Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0            0           0                    0                 0   
1            0           0                    0                 0   
2            0           0                    0                 0   
3            0           0                    0                 0   
4            0           0                    0                 0   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                   0              0          0           0           0   
1                   0              0          0           0           0   
2                   0              0          1           0           0   
3                   0              0          0           0           0   
4                   0              0          0           0           0   

   Bakery  Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  Beer Bar  \
0       0     0    0                 0                   0      0         0   
1       0     0    0                 0                   0      0         0   
2       0     0    0                 0                   0      0         0   
3       0     0    0                 0                   0      0         0   
4       0     0    0                 0                   0      0         0   

   Beer Store  Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Boutique  \
0           0                   0       0              0          0         0   
1           0                   0       0              0          0         0   
2           0                   0       0              0          0         0   
3           0                   0       0              0          0         0   
4           0                   0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Building  \
0                     0               0        0                0         0   
1                     0               0        0                0         0   
2                     0               0        0                0         0   
3                     0               0        0                0         0   
4                     0               0        0                0         0   

 

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

Neighbourhood  Accessories Store  \
0                           Adelaide, King, Richmond               0.01   
1                                        Berczy Park               0.00   
2       Brockton, Exhibition Place, Parkdale Village               0.00   
3   Business reply mail Processing Centre969 Eastern               0.00   
4  CN Tower, Bathurst Quay, Island airport, Harbo...               0.00   

   Adult Boutique  Afghan Restaurant   Airport  Airport Food Court  \
0             0.0                0.0  0.000000            0.000000   
1             0.0                0.0  0.000000            0.000000   
2             0.0                0.0  0.000000            0.000000   
3             0.0                0.0  0.000000            0.000000   
4             0.0                0.0  0.071429            0.071429   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0      0.000000        0.000000         0.000000          0.000000   
1      0.000000        0.000000         0.000000          0.000000   
2      0.000000        0.000000         0.000000          0.000000   
3      0.000000        0.000000         0.000000          0.000000   
4      0.071429        0.142857         0.142857          0.142857   

   American Restaurant  Antique Shop  Aquarium  Arepa Restaurant  \
0                 0.04           0.0       0.0               0.0   
1                 0.00           0.0       0.0               0.0   
2                 0.00           0.0       0.0               0.0   
3                 0.00           0.0       0.0               0.0   
4                 0.00           0.0       0.0               0.0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                     0.0     0.010000        0.01                  0.0   
1                     0.0     0.017857        0.00                  0.0   
2                     0.0     0.000000        0.00                  0.0   
3                     0.0     0.000000        0.00                  0.0   
4                     0.0     0.000000        0.00                  0.0   

   Asian Restaurant  Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  \
0              0.02                 0.0       0.000000        0.0         0.0   
1              0.00                 0.0       0.000000        0.0         0.0   
2              0.00                 0.0       0.000000        0.0         0.0   
3              0.00                 0.0       0.055556        0.0         0.0   
4              0.00                 0.0       0.000000        0.0         0.0   

   Bagel Shop    Bakery  Bank   Bar  Baseball Stadium  Basketball Stadium  \
0    0.000000  0.020000   0.0  0.03               0.0            0.000000   
1    0.017857  0.035714   0.0  0.00               0.0            0.017857   
2    0.000000  0.000000   0.0  0.05               0.0            0.000000   
3    0.000000  0.000000   0.0  0.00               0.0            0.000000   
4    0.000000  0.000000   0.0  0.00               0.0            0.000000   

      Beach  Beer Bar  Beer Store  Belgian Restaurant    Bistro  \
0  0.000000  0.000000         0.0            0.000000  0.000000   
1  0.017857  0.017857         0.0            0.017857  0.017857   
2  0.000000  0.000000         0.0            0.000000  0.000000   
3  0.000000  0.000000         0.0            0.000000  0.000000   
4  0.000000  0.000000         0.0            0.000000  0.000000   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0       0.000000       0.01  0.000000                  0.01            0.03   
1       0.000000       0.00  0.000000                  0.00            0.00   
2       0.000000       0.00  0.000000                  0.00            0.10   
3       0.000000       0.00  0.000000                  0.00            0.00   
4       0.071429       0.00  0.071429                  0.00            0.00   

    Brewery  Bubble Tea Shop  Building  Burger Joint  Burr

In [57]:
toronto_grouped.shape

(38, 242)

#### Let's print each neighborhood along with the top 5 most common venues

In [60]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print("---"+hood+'----')
    temp = toronto_grouped[toronto_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4      Thai Restaurant  0.04


---Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2      Restaurant  0.05
3  Farmers Market  0.04
4             Pub  0.04


---Brockton, Exhibition Place, Parkdale Village----
                  venue  freq
0           Coffee Shop  0.15
1        Breakfast Spot  0.10
2                  Café  0.10
3                   Gym  0.05
4  Caribbean Restaurant  0.05


---Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Pizza Place  0.06
2          Restaurant  0.06
3             Brewery  0.06
4          Skate Park  0.06


---CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1

#### Let's put that into a pandas dataframe

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighbourhood  \
0                            Adelaide, King, Richmond   
1                                         Berczy Park   
2        Brockton, Exhibition Place, Parkdale Village   
3    Business reply mail Processing Centre969 Eastern   
4   CN Tower, Bathurst Quay, Island airport, Harbo...   
5                         Cabbagetown, St. James Town   
6                                  Central Bay Street   
7           Chinatown, Grange Park, Kensington Market   
8                                            Christie   
9                                Church and Wellesley   
10                     Commerce Court, Victoria Hotel   
11                                         Davisville   
12                                   Davisville North   
13  Deer Park, Forest Hill SE, Rathnelly, South Hi...   
14           Design Exchange, Toronto Dominion Centre   
15                       Dovercourt Village, Dufferin   
16             First Canadian Place, Underground city   
17                Forest Hill North, Forest Hill West   
18                     Harbord, University of Toronto   
19  Harbourfront East, Toronto Islands, Union Station   
20                          Harbourfront, Regent Park   
21                      High Park, The Junction South   
22                                      Lawrence Park   
23                           Little Portugal, Trinity   
24                        Moore Park, Summerhill East   
25                                 North Toronto West   
26                             Parkdale, Roncesvalles   
27                                           Rosedale   
28                                           Roselawn   
29                                 Runnymede, Swansea   
30                           Ryerson, Garden District   
31                                     St. James Town   
32                    Stn A PO Boxes 25 The Esplanade   
33                                    Studio District   
34                The Annex, North Midtown, Yorkville   
35                                        The Beaches   
36                     The Beaches West, India Bazaar   
37                       The Danforth West, Riverdale   

      1st Most Common Venue 2nd Most Common Venue  \
0               Coffee Shop                  Café   
1               Coffee Shop            Restaurant   
2               Coffee Shop        Breakfast Spot   
3        Light Rail Station            Comic Shop   
4            Airport Lounge      Airport Terminal   
5               Coffee Shop            Restaurant   
6               Coffee Shop                  Café   
7                      Café                   Bar   
8                      Café         Grocery Store   
9       Japanese Restaurant           Coffee Shop   
10              Coffee Shop                  Café   
11           Sandwich Place          Dessert Shop   
12                    Hotel     Food & Drink Shop   
13              Coffee Shop                   Pub   
14              Coffee Shop                  Café   
15           Discount Store           Supermarket   
16              Coffee Shop                  Café   
17         Sushi Restaurant                  Park   
18                Bookstore                  Café   
19              Coffee Shop                 Hotel   
20              Coffee Shop                Bakery   
21       Mexican Restaurant                  Café   
22       Dim Sum Restaurant           Swim School   
23                      Bar                  Café   
24             Intersection            Playground   
25           Clothing Store   Sporting Goods Shop   
26           Breakfast Spot             Gift Shop   
27                     Park                 Trail   
28  Health & Beauty Service                Garden   
29              Coffee Shop           Pizza Place   
30              Coffee Shop        Clothing Store   
31              Coffee Shop                  Café   
32              Coffee Shop                  Café   
33            

## Cluster Neighbourhoods

In [67]:
# set number of clusters
k_clusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood',1)
#print(toronto_grouped_clustering.head())
kmeans = KMeans(n_clusters=k_clusters,random_state=0).fit(toronto_grouped_clustering)

a = kmeans.labels_
a

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0], dtype=int32)

In [68]:
toronto_grouped_clustering.head()

Accessories Store  Adult Boutique  Afghan Restaurant   Airport  \
0               0.01             0.0                0.0  0.000000   
1               0.00             0.0                0.0  0.000000   
2               0.00             0.0                0.0  0.000000   
3               0.00             0.0                0.0  0.000000   
4               0.00             0.0                0.0  0.071429   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0            0.000000      0.000000        0.000000         0.000000   
1            0.000000      0.000000        0.000000         0.000000   
2            0.000000      0.000000        0.000000         0.000000   
3            0.000000      0.000000        0.000000         0.000000   
4            0.071429      0.071429        0.142857         0.142857   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  \
0          0.000000                 0.04           0.0       0.0   
1          0.000000                 0.00           0.0       0.0   
2          0.000000                 0.00           0.0       0.0   
3          0.000000                 0.00           0.0       0.0   
4          0.142857                 0.00           0.0       0.0   

   Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
0               0.0                     0.0     0.010000        0.01   
1               0.0                     0.0     0.017857        0.00   
2               0.0                     0.0     0.000000        0.00   
3               0.0                     0.0     0.000000        0.00   
4               0.0                     0.0     0.000000        0.00   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                  0.0              0.02                 0.0       0.000000   
1                  0.0              0.00                 0.0       0.000000   
2                  0.0              0.00                 0.0       0.000000   
3                  0.0              0.00                 0.0       0.055556   
4                  0.0              0.00                 0.0       0.000000   

   BBQ Joint  Baby Store  Bagel Shop    Bakery  Bank   Bar  Baseball Stadium  \
0        0.0         0.0    0.000000  0.020000   0.0  0.03               0.0   
1        0.0         0.0    0.017857  0.035714   0.0  0.00               0.0   
2        0.0         0.0    0.000000  0.000000   0.0  0.05               0.0   
3        0.0         0.0    0.000000  0.000000   0.0  0.00               0.0   
4        0.0         0.0    0.000000  0.000000   0.0  0.00               0.0   

   Basketball Stadium     Beach  Beer Bar  Beer Store  Belgian Restaurant  \
0            0.000000  0.000000  0.000000         0.0            0.000000   
1            0.017857  0.017857  0.017857         0.0            0.017857   
2            0.000000  0.000000  0.000000         0.0            0.000000   
3            0.000000  0.000000  0.000000         0.0            0.000000   
4            0.000000  0.000000  0.000000         0.0            0.000000   

     Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0  0.000000       0.000000       0.01  0.000000                  0.01   
1  0.017857       0.000000       0.00  0.000000                  0.00   
2  0.000000       0.000000       0.00  0.000000                  0.00   
3  0.000000       0.000000       0.00  0.000000                  0.00   
4  0.000000       0.071429       0.00  0.071429                  0.00   

   Breakfast Spot   Brewery  Bubble Tea Shop  Building  Burger Joint  \
0            0.03  0.000000              0.0      0.01          0.02   
1            0.00  0.000000              0.0      0.00          0.00   
2            0.10  0.000000              0.0      0.00          0.00   
3            0.00  0.055556              0.0      0.00          0.00   
4            0.00  0.000000              0.0      0.00          0.00   

   Burrito Place  Bus 

let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [70]:
toronto_merged = df
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'),on='Neighbourhood')
toronto_merged.head()

Postcode          Borough                   Neighbourhood   Latitude  \
37      M4E     East Toronto                     The Beaches  43.676357   
41      M4K     East Toronto    The Danforth West, Riverdale  43.679557   
42      M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
43      M4M     East Toronto                 Studio District  43.659526   
44      M4N  Central Toronto                   Lawrence Park  43.728020   

    Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
37 -79.293031               0          Neighborhood                   Pub   
41 -79.352188               0      Greek Restaurant        Ice Cream Shop   
42 -79.315572               0                  Park        Ice Cream Shop   
43 -79.340923               0                  Café           Coffee Shop   
44 -79.388790               0    Dim Sum Restaurant           Swim School   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
37           Coffee Shop             BBQ Joint                 Diner   
41           Coffee Shop             Bookstore    Italian Restaurant   
42             Pet Store           Pizza Place                   Pub   
43                Bakery             Gastropub    Italian Restaurant   
44                  Park         Jewelry Store              Bus Line   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
37        Discount Store               Dog Run      Doner Restaurant   
41         Grocery Store               Brewery       Bubble Tea Shop   
42         Movie Theater         Burrito Place          Burger Joint   
43   American Restaurant           Yoga Studio       Coworking Space   
44           Yoga Studio            Donut Shop               Dog Run   

   9th Most Common Venue       10th Most Common Venue  
37            Donut Shop                  Yoga Studio  
41            Restaurant                         Café  
42        Sandwich Place                      Brewery  
43               Brewery               Sandwich Place  
44      Doner Restaurant  Eastern European Restaurant

Finally, let's visualize the resulting clusters.

In [76]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Eaxm Clusters

##### cluster 1

In [108]:
cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels']==0,toronto_merged.columns[[2]+list(range(5,toronto_merged.shape[1]))]]
cluster_1

Neighbourhood  Cluster Labels  \
37                                        The Beaches               0   
41                       The Danforth West, Riverdale               0   
42                     The Beaches West, India Bazaar               0   
43                                    Studio District               0   
44                                      Lawrence Park               0   
45                                   Davisville North               0   
46                                 North Toronto West               0   
47                                         Davisville               0   
48                        Moore Park, Summerhill East               0   
49  Deer Park, Forest Hill SE, Rathnelly, South Hi...               0   
50                                           Rosedale               0   
51                        Cabbagetown, St. James Town               0   
52                               Church and Wellesley               0   
53                          Harbourfront, Regent Park               0   
54                           Ryerson, Garden District               0   
55                                     St. James Town               0   
56                                        Berczy Park               0   
58                           Adelaide, King, Richmond               0   
59  Harbourfront East, Toronto Islands, Union Station               0   
60           Design Exchange, Toronto Dominion Centre               0   
61                     Commerce Court, Victoria Hotel               0   
63                                           Roselawn               0   
64                Forest Hill North, Forest Hill West               0   
66                     Harbord, University of Toronto               0   
67          Chinatown, Grange Park, Kensington Market               0   
70             First Canadian Place, Underground city               0   
75                                           Christie               0   
76                       Dovercourt Village, Dufferin               0   
77                           Little Portugal, Trinity               0   
78       Brockton, Exhibition Place, Parkdale Village               0   
82                      High Park, The Junction South               0   
84                                 Runnymede, Swansea               0   
87   Business reply mail Processing Centre969 Eastern               0   

      1st Most Common Venue 2nd Most Common Venue  \
37             Neighborhood                   Pub   
41         Greek Restaurant        Ice Cream Shop   
42                     Park        Ice Cream Shop   
43                     Café           Coffee Shop   
44       Dim Sum Restaurant           Swim School   
45                    Hotel     Food & Drink Shop   
46           Clothing Store   Sporting Goods Shop   
47           Sandwich Place          Dessert Shop   
48             Intersection            Playground   
49              Coffee Shop                   Pub   
50                     Park                 Trail   
51              Coffee Shop            Restaurant   
52      Japanese Restaurant           Coffee Shop   
53              Coffee Shop                Bakery   
54              Coffee Shop        Clothing Store   
55              Coffee Shop                  Café   
56              Coffee Shop            Restaurant   
58              Coffee Shop                  Café   
59              Coffee Shop                 Hotel   
60              Coffee Shop                  Café   
61              Coffee Shop                  Café   
63  Health & Beauty Service                Garden   
64         Sushi Restaurant                  Park   
66                Bookstore                  Café   
67                     Café                   Bar   
70              Coffee Shop                  Café   
75                     Café         Grocery Store   
76           Discount Store           Supermarket   
77                  

##### cluster 2

In [109]:
cluster_2 = toronto_merged.loc[toronto_merged['Cluster Labels']==1,toronto_merged.columns[[2]+list(range(5,toronto_merged.shape[1]))]]
cluster_2

Neighbourhood  Cluster Labels  \
65                The Annex, North Midtown, Yorkville               1   
68  CN Tower, Bathurst Quay, Island airport, Harbo...               1   

   1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
65           Coffee Shop                  Café        Sandwich Place   
68        Airport Lounge      Airport Terminal       Airport Service   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
65           Pizza Place              Pharmacy     Indian Restaurant   
68                 Plane      Sculpture Garden              Boutique   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
65     Convenience Store        Cosmetics Shop                   Pub   
68         Boat or Ferry       Harbor / Marina          Airport Gate   

   10th Most Common Venue  
65           Burger Joint  
68                Airport

###### clusters 3

In [110]:
cluster_3 = toronto_merged.loc[toronto_merged['Cluster Labels']==2,toronto_merged.columns[[2]+list(range(5,toronto_merged.shape[1]))]]
cluster_3

Neighbourhood  Cluster Labels 1st Most Common Venue  \
69  Stn A PO Boxes 25 The Esplanade               2           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
69                  Café            Restaurant    Seafood Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
69                   Pub          Cocktail Bar                 Hotel   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
69                   Gym           Cheese Shop            Art Gallery

###### cluster 4

In [111]:
cluster_4 = toronto_merged.loc[toronto_merged['Cluster Labels']==3,toronto_merged.columns[[2]+list(range(5,toronto_merged.shape[1]))]]
cluster_4

Neighbourhood  Cluster Labels 1st Most Common Venue  \
57  Central Bay Street               3           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
57                  Café    Italian Restaurant                   Bar   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
57        Sandwich Place   Japanese Restaurant          Burger Joint   

   8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
57        Ice Cream Shop                   Spa  Middle Eastern Restaurant

##### cluster 5

In [112]:
cluster_5 = toronto_merged.loc[toronto_merged['Cluster Labels']==4,toronto_merged.columns[[2]+list(range(5,toronto_merged.shape[1]))]]
cluster_5

Neighbourhood  Cluster Labels 1st Most Common Venue  \
83  Parkdale, Roncesvalles               4        Breakfast Spot   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
83             Gift Shop             Bookstore         Movie Theater   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
83            Restaurant    Italian Restaurant          Dessert Shop   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
83          Burger Joint                   Bar  Eastern European Restaurant